<a href="https://colab.research.google.com/github/kuakeke/113-2programming/blob/main/2025%E5%B9%B44%E6%9C%8824%E6%97%A5%20%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%20%E4%BB%BB%E5%8B%99%20%E5%BE%9E%E7%95%99%E8%A8%80%E4%B8%AD%E6%8A%93%E5%87%BA%E9%97%9C%E9%8D%B5%E5%AF%A6%E9%AB%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

柯昱葶 國文115級 41120122L

# 🔍 任務背景
本任務目標是練習從自然語言中提取關鍵資訊。

# 🎯 任務要求

## 1️⃣ 準備與讀入資料
* 從網路上搜尋一則你感興趣的真實討論資料（例如：Dcard、PTT、新聞留言）。

* 複製 5～10 則留言或段落，整理後儲存為 comments.csv，欄位包含：

 * Text（留言內容）

## 2️⃣ 定義要提取的實體類別
* 至少指定三類實體（可參考）：

 * 👤 人名（Person）

 * 🏢 單位名稱（Organization）

 * 📍 地點名稱（Location）

## 3️⃣ 中文文字前處理與實體抽取
* 使用 jieba 斷詞。

* 利用 關鍵詞比對 或 正則表達式，從每段留言中提取上述實體。

* 請用以下格式儲存提取結果：

 * [

  {"text": "台灣大學", "type": "Organization"},

  {"text": "台北", "type": "Location"},

  {"text": "李四", "type": "Person"}

]


## 4️⃣ 詞頻統計與視覺化（選做加分）
* 統計各類實體的出現次數。

* 視覺化為長條圖或詞雲圖。

**🛠️ 工具建議**

* pandas、jieba、re

* collections.Counter

* matplotlib / plotly / wordcloud（選做加分）

**💡 額外挑戰（選做加分）**

* 自建實體詞庫強化抽取效果。

* 將結果存成 entities.csv，供後續分析使用。

* 嘗試用 Gemini API 幫忙做命名實體辨識，並與手動提取結果比較。



---



Dcard原文：https://www.dcard.tw/f/mood/p/258287882

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Deqdck7cu3ft9L3zWhzBr3BywXuLnj03a6PYn6j-0Qo/edit?usp=sharing').sheet1

In [3]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Deqdck7cu3ft9L3zWhzBr3BywXuLnj03a6PYn6j-0Qo/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [4]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.head()

,Text
0,景文科技大學，男，為什麼被打的不碼，碼打人的
1,元智大學，男，看髮型跟穿搭就知道是系排墊底菸猴\n唯一的價值就是繳學費給認真同學的獎學金
2,國立成功大學，男，我寧願把我的稅金拿去餵狗
3,國立臺北科技大學，女，辛苦辛苦的讀書當上老師，結果只能教猴子，還不能真的教，教了會被打，真令...
4,龍華科技大學，男，這沒退學 我就把畢業證書撕了\n老師辛苦了


In [6]:
!pip install jieba
!pip install snownlp

In [7]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [8]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [9]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [10]:
df['Text'] = df['Text'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Text'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 1.566 seconds.
DEBUG:jieba:Loading model cost 1.566 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [11]:
df['Tokenized']

,Tokenized
0,"[景文, 科技, 大學, 男為, 什麼, 被, 打, 的, 不, 碼碼, 打人, 的]"
1,"[元智大學, 男, 看, 髮, 型, 跟, 穿, 搭, 就, 知道, 是, 系排, 墊底, ..."
2,"[國立, 成功, 大學, 男, 我寧願, 把, 我, 的, 稅金, 拿, 去, 餵, 狗]"
3,"[國立, 臺, 北, 科技, 大學女, 辛苦, 辛苦, 的, 讀書, 當上, 老師, 結果,..."
4,"[龍華, 科技, 大學, 男, 這沒, 退學, 我, 就, 把, 畢業, 證書, 撕, 了,..."
5,"[龍華, 科技, 大學女, 教授, 有夠, 可憐教, 猴子, 還得, 被, 猴子, 打]"
6,"[國立, 中興大學, 女先, 不, 說, 誰, 對, 誰, 錯, 畢竟, 一個, 影片, 看..."
7,"[國立, 臺, 中, 教育, 大學女, 看, 了, 本人, 發的文, 只能, 說現, 在, ..."
8,"[國立, 臺, 北, 科技, 大學, 男, 這裡, 最大, 的, 問題, 不是, 他, 帶,..."
9,"[國立, 臺, 灣師, 範大學, 男, 你, 知道, 私立, 科大, 學生, 最大, 嗎, ..."


In [12]:
import re
import pandas as pd

# 定義男女函數
def gender(row):
    if any(keyword in row['Text'] for keyword in ['男']):
        return '男'
    elif any(keyword in row['Text'] for keyword in ['女']):
        return '女'
    else:
        return '未知'

# 定義學校提取函數
def extract_school(text):
    # 正則表達式匹配學校名稱
    school_keywords = r'(大學)'
    match = re.search(r'([\u4e00-\u9fa5]+?' + school_keywords + ')', text)
    if match:
        school = match.group(0)
        # 截取最後八個字作為學校名稱
        return school[-8:]
    return None

# 定義內文提及關鍵字“猴"和“教授"函數
def nickname(row):
    if any(keyword in row['Text'] for keyword in ['猴', '猴子']):
        return '猴子'
    elif any(keyword in row['Text'] for keyword in ['教授']):
        return '教授'
    elif any(keyword in row['Text'] for keyword in ['學生', '同學']):
        return '學生'
    else:
        return None

# 定義提取內文的函數
def extract_content(text):
    school = extract_school(text)
    if school:
        text = text.replace(school, '')
    gender_keywords = r'(男|女)'
    text = re.sub(gender_keywords, '', text)
    content = re.sub(r'\s+', ' ', text).strip()
    return content

# 應用函數到數據
df['Gender'] = df.apply(gender, axis=1)
df['School'] = df['Text'].apply(lambda x: extract_school(str(x)))
df['Nickname'] = df.apply(nickname, axis=1)
df['Content'] = df['Text'].apply(extract_content)


In [13]:
print(df[['Text', 'Gender', 'School', 'Nickname', 'Content']])

                                                Text Gender    School  \
0                                景文科技大學男為什麼被打的不碼碼打人的      男    景文科技大學   
1           元智大學男看髮型跟穿搭就知道是系排墊底菸猴唯一的價值就是繳學費給認真同學的獎學金      男      元智大學   
2                                國立成功大學男我寧願把我的稅金拿去餵狗      男    國立成功大學   
3  國立臺北科技大學女辛苦辛苦的讀書當上老師結果只能教猴子還不能真的教教了會被打真令人心痛不想唸...      女  國立臺北科技大學   
4                          龍華科技大學男這沒退學我就把畢業證書撕了老師辛苦了      男    龍華科技大學   
5                             龍華科技大學女教授有夠可憐教猴子還得被猴子打      女    龍華科技大學   
6  國立中興大學女先不說誰對誰錯畢竟一個影片看不出來這麼多細節但那位同學頭髮染成這樣真的不會有人...      女    國立中興大學   
7  國立臺中教育大學女看了本人發的文只能說現在當老師好可悲學生遲到進入教室帶食物容器進電腦教室真...      女  國立臺中教育大學   
8  國立臺北科技大學男這裡最大的問題不是他帶飲料或是打人最大的問題是他根本不覺得自己有錯還要狡辯...      男  國立臺北科技大學   
9             國立臺灣師範大學男你知道私立科大學生最大嗎得罪學生學費怎麼辦教授的薪水怎麼辦      男  國立臺灣師範大學   

  Nickname                                            Content  
0     None                                       為什麼被打的不碼碼打人的  
1       猴子                看髮型跟穿搭就知道是系排墊底菸猴唯一的價值就是繳學費給認真同學的獎學金  
2   

In [14]:
df.head()

,Text,Tokenized,Gender,School,Nickname,Content
0,景文科技大學男為什麼被打的不碼碼打人的,"[景文, 科技, 大學, 男為, 什麼, 被, 打, 的, 不, 碼碼, 打人, 的]",男,景文科技大學,None,為什麼被打的不碼碼打人的
1,元智大學男看髮型跟穿搭就知道是系排墊底菸猴唯一的價值就是繳學費給認真同學的獎學金,"[元智大學, 男, 看, 髮, 型, 跟, 穿, 搭, 就, 知道, 是, 系排, 墊底, ...",男,元智大學,猴子,看髮型跟穿搭就知道是系排墊底菸猴唯一的價值就是繳學費給認真同學的獎學金
2,國立成功大學男我寧願把我的稅金拿去餵狗,"[國立, 成功, 大學, 男, 我寧願, 把, 我, 的, 稅金, 拿, 去, 餵, 狗]",男,國立成功大學,None,我寧願把我的稅金拿去餵狗
3,國立臺北科技大學女辛苦辛苦的讀書當上老師結果只能教猴子還不能真的教教了會被打真令人心痛不想唸...,"[國立, 臺, 北, 科技, 大學女, 辛苦, 辛苦, 的, 讀書, 當上, 老師, 結果,...",女,國立臺北科技大學,猴子,辛苦辛苦的讀書當上老師結果只能教猴子還不能真的教教了會被打真令人心痛不想唸書真的不需要去學校...
4,龍華科技大學男這沒退學我就把畢業證書撕了老師辛苦了,"[龍華, 科技, 大學, 男, 這沒, 退學, 我, 就, 把, 畢業, 證書, 撕, 了,...",男,龍華科技大學,None,這沒退學我就把畢業證書撕了老師辛苦了


In [15]:
# 統計性別分布
print("性別統計：")
print(df['Gender'].value_counts(), end="\n\n")

# 統計學校出現次數
print("學校出現次數：")
print(df['School'].value_counts(), end="\n\n")

# 統計暱稱/角色分布
print("暱稱統計：")
print(df['Nickname'].value_counts(), end="\n\n")


性別統計：
Gender
男    6
女    4
Name: count, dtype: int64

學校出現次數：
School
國立臺北科技大學    2
龍華科技大學      2
元智大學        1
景文科技大學      1
國立成功大學      1
國立中興大學      1
國立臺中教育大學    1
國立臺灣師範大學    1
Name: count, dtype: int64

暱稱統計：
Nickname
猴子    5
學生    1
教授    1
Name: count, dtype: int64



In [16]:
from collections import Counter

school_counts = Counter(df['School'])

In [17]:
school_counts

Counter({'景文科技大學': 1,
         '元智大學': 1,
         '國立成功大學': 1,
         '國立臺北科技大學': 2,
         '龍華科技大學': 2,
         '國立中興大學': 1,
         '國立臺中教育大學': 1,
         '國立臺灣師範大學': 1})

In [18]:
import plotly.express as px

# Sorting the school counts in descending order
sorted_school_counts = sorted(school_counts.items(), key=lambda x: x[1], reverse=True)
sorted_school_counts_dict = dict(sorted_school_counts)

# Creating a sorted DataFrame for the bar chart
sorted_school_df = pd.DataFrame(list(sorted_school_counts_dict.items()), columns=["School", "Count"])

# Plotting the sorted school distribution
fig_sorted = px.bar(sorted_school_df, x='School', y='Count',
                    title="School Distribution (Sorted by Count)",
                    labels={"School": "School", "Count": "Number of Mentions"},
                    text='Count')

fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')
fig_sorted.update_layout(xaxis=dict(title="School"), yaxis=dict(title="Number of Mentions"))

fig_sorted.show()

In [19]:
import pandas as pd
import plotly.express as px

# 統計 Nickname 出現次數
nickname_counts = df['Nickname'].value_counts().reset_index()
nickname_counts.columns = ['Nickname', 'Frequency']

# 畫 Plotly 長條圖
fig = px.bar(
    nickname_counts, x='Nickname', y='Frequency',
    title='Nickname 出現次數統計',
    labels={'Nickname': '暱稱', 'Frequency': '出現次數'},
    text='Frequency'
)

# 美化版面
fig.update_layout(
    xaxis_title="暱稱",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()


整理文字欄位，並導進第三個GoogleSheet裡面，以方便儲存使用。

In [20]:
df_set = df[['School', 'Gender', 'Nickname', 'Content']]

In [21]:
df_set

,School,Gender,Nickname,Content
0,景文科技大學,男,None,為什麼被打的不碼碼打人的
1,元智大學,男,猴子,看髮型跟穿搭就知道是系排墊底菸猴唯一的價值就是繳學費給認真同學的獎學金
2,國立成功大學,男,None,我寧願把我的稅金拿去餵狗
3,國立臺北科技大學,女,猴子,辛苦辛苦的讀書當上老師結果只能教猴子還不能真的教教了會被打真令人心痛不想唸書真的不需要去學校...
4,龍華科技大學,男,None,這沒退學我就把畢業證書撕了老師辛苦了
5,龍華科技大學,女,猴子,教授有夠可憐教猴子還得被猴子打
6,國立中興大學,女,學生,先不說誰對誰錯畢竟一個影片看不出來這麼多細節但那位同學頭髮染成這樣真的不會有人覺得他是一個來...
7,國立臺中教育大學,女,猴子,看了本人發的文只能說現在當老師好可悲學生遲到進入教室帶食物容器進電腦教室真的沒有人管你有沒有...
8,國立臺北科技大學,男,猴子,這裡最大的問題不是他帶飲料或是打人最大的問題是他根本不覺得自己有錯還要狡辯老師先動手他只是還...
9,國立臺灣師範大學,男,教授,你知道私立科大學生最大嗎得罪學生學費怎麼辦教授的薪水怎麼辦


In [22]:
from gspread_dataframe import set_with_dataframe

gc = gspread.authorize(creds)
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Deqdck7cu3ft9L3zWhzBr3BywXuLnj03a6PYn6j-0Qo/edit?usp=sharing')

# 選擇第三個工作表
worksheet = spreadsheet.get_worksheet(2)

# 將 DataFrame 寫入第二個工作表
set_with_dataframe(worksheet, df_set)

用Gemini llm 分析事件和比較數據結果。

In [23]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [24]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"用中文分析社交媒體上對特定事件的風向：{df_set.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這份社交媒體資料反映了對某起校園暴力事件的強烈負面情緒，主要針對施暴學生。分析如下：

**主要風向：**  輿論一邊倒地譴責施暴學生，將其形容為「猴子」，並對台灣教育體制和部分家長教育方式表示失望和批評。

**具體表現：**

* **施暴學生被強烈負面標籤化：**  「猴子」一詞被大量使用，帶有極強的貶義，暗示施暴學生缺乏教養、行為粗魯、智力低下。這種標籤化言論展現了網友們的憤怒和鄙視。

* **對老師的同情與支持：**  多數評論表達了對被打老師的同情，認為老師工作辛苦，卻要面對如此不尊師重道的學生，令人心痛。這反映了社會大眾對教師職業的關注和尊重。

* **對教育體制的質疑與批判：**  部分評論指責台灣的教育體制和家庭教育的缺失，認為這些因素導致學生缺乏基本道德和規範意識，才會出現如此惡劣的事件。

* **對私立科大的負面刻板印象：**  最後一條評論反映了部分網友對私立科技大學學生的負面刻板印象，認為他們「最大」，暗示其行為囂張跋扈，缺乏約束。這屬於偏見性言論，缺乏客觀性。

* **缺乏事件細節，以情緒性言論為主：**  評論大多基於片段資訊和個人情緒，缺乏對事件經過的詳細描述和客觀分析，很多言論帶有過激情緒。

**總結：**

社交媒體上的風向顯示，這起校園暴力事件引發了廣泛的社會關注和強烈反響。輿論一邊倒地譴責施暴學生，同時也反思了教育體制和家庭教育的問題。  然而，由於資訊不完整，評論多以情緒性言論為主，缺乏對事件本身的全面客觀分析，容易造成偏見和誤解。要更全面地理解事件，需要更多客觀的資訊和調查結果。

